<a href="https://colab.research.google.com/github/TheLadyJack/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE

In [1]:
# Grab the file from UCI
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-02-17 20:44:04--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-02-17 20:44:04 (285 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
from scipy import stats

In [2]:
import pandas as pd

df = pd.read_csv('house-votes-84.data', names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [4]:
import numpy as np
df = df.replace({"y": 1, "n": 0, "?": np.NaN})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [5]:
rep = df[df['party']=='republican']

rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [6]:
dem = df[df['party']=='democrat']

dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [12]:
stats.ttest_ind(dem['aid-to-contras'], rep['aid-to-contras'], nan_policy='omit')

Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

100% I expected the republicans to support this bill over the democrats, but the results are that a democrat is more likely to to support this bill with a p-value of functionally zero.

In [13]:
stats.ttest_ind(rep['education'], dem['education'], nan_policy='omit')

Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

I could be lazy and use this example for a bill with a large republican partisanship, but it was used in class, so instead I'll use

In [15]:
stats.ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit')

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

Holy Canoli!

That is some partisanship right there!

Wow!!!

that's the lowest p-value I've seen yet,
and in favor of rep support of a weird bill to boot

In [16]:
print(rep['physician-fee-freeze'].mean())
print(dem['physician-fee-freeze'].mean())

0.9878787878787879
0.05405405405405406


In [17]:
stats.ttest_ind(rep['budget'], dem['budget'], nan_policy='omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

wow! i expected that to be closer

In [19]:
dem.describe() - rep.describe()

,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
count,93.000000,91.000000,96.000000,94.000000,90.000000,92.000000,97.000000,106.000000,83.000000,98.000000,96.000000,94.000000,94.000000,96.000000,95.000000,39.000000
mean,0.416772,-0.004665,0.754315,-0.933825,-0.735829,-0.420846,0.531460,0.676031,0.642913,-0.086093,0.373807,-0.726389,-0.571077,-0.631172,0.547707,0.277601
std,0.098071,-0.000607,-0.026448,0.116802,0.196664,0.196326,-0.008634,0.016307,0.108945,0.001952,0.161306,0.016061,0.107220,0.342313,0.194962,-0.229212
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,1.000000
50%,1.000000,0.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,-1.000000,-1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,1.000000,0.000000
max,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Should I have done this from the get-go? 
probably, yes.
but it looks like the water project is our go-to bi-partisan issue

immigration is a surprising second place

In [20]:
stats.ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

And there we have it. Water comes in with a whopping p-value of 92.9%

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
def double_tstat (var1, var2):
  if stats.ttest_ind(var1, var2, nan_policy='omit')[1] < 0.05:
    if stats.ttest_ind(var1, var2, nan_policy='omit').statistic > 0:
      print("var1 being true is significantly more likely than var2.")
    else:
      print("var2 being true is significantly more likely than var1.")
  else:
    print("var1 and var2 are statistically similar in this regard.")

In [36]:
double_tstat(rep['budget'], dem['budget'])

var2 being true is significantly more likely than var1.


In [29]:
stats.ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')[0]

0.08896538137868286

In [39]:
double_tstat(rep['water-project'], dem['water-project'])

var1 and var2 are statistically similar in this regard.


In [40]:
!wget https://www.bjs.gov/content/pub/sheets/hpnvv0812.zip

--2020-02-17 23:40:06--  https://www.bjs.gov/content/pub/sheets/hpnvv0812.zip
Resolving www.bjs.gov (www.bjs.gov)... 149.101.82.172, 2607:f330:5fa1:1020::ac
Connecting to www.bjs.gov (www.bjs.gov)|149.101.82.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22286 (22K) [application/zip]
Saving to: ‘hpnvv0812.zip’

hpnvv0812.zip       100%[===================>]  21.76K  --.-KB/s    in 0.01s   

2020-02-17 23:40:06 (1.43 MB/s) - ‘hpnvv0812.zip’ saved [22286/22286]



In [42]:
!unzip hpnvv0812.zip


Archive:  hpnvv0812.zip
  inflating: readme.txt              
  inflating: hpnvv0812t05.csv        
  inflating: hpnvv0812t04.csv        
  inflating: hpnvv0812t03.csv        
  inflating: hpnvv0812t02.csv        
  inflating: hpnvv0812t01.csv        
  inflating: hpnvv0812f09.csv        
  inflating: hpnvv0812f08.csv        
  inflating: hpnvv0812f07.csv        
  inflating: hpnvv0812f06.csv        
  inflating: hpnvv0812f05.csv        
  inflating: hpnvv0812f04.csv        
  inflating: hpnvv0812f03.csv        
  inflating: hpnvv0812f02.csv        
  inflating: hpnvv0812f01.csv        
  inflating: hpnvv0812at13.csv       
  inflating: hpnvv0812at12.csv       
  inflating: hpnvv0812at11.csv       
  inflating: hpnvv0812at10.csv       
  inflating: hpnvv0812at09.csv       
  inflating: hpnvv0812at08.csv       
  inflating: hpnvv0812at07.csv       
  inflating: hpnvv0812at06.csv       
  inflating: hpnvv0812at05.csv       
  inflating: hpnvv0812at04.csv       
  inflating: hpnvv0812at03

In [47]:
!curl hpnvv0812at01.csv

curl: (6) Could not resolve host: hpnvv0812at01.csv


In [0]:
#df = pd.read_csv('hpnvv0812at01.csv')
#df.head()

#This is the data that I want to use for unit 1 project, 
#but I seem to be having some difficulty in opening it at the moment.
#I'll come back to this, but for now

In [77]:
df = pd.read_csv('https://raw.githubusercontent.com/Lord-Kanzler/DS-Unit-1-Sprint-2-Statistics/master/module1/Tuna.csv')

df.head(100)

,Tuna,Mercury
0,albacore,0.000
1,albacore,0.410
2,albacore,0.820
3,albacore,0.320
4,albacore,0.036
...,...,...
95,yellowfin,0.327
96,yellowfin,0.176
97,yellowfin,0.169
98,yellowfin,0.293


In [82]:
df.Tuna[1]

'albacore '

In [83]:

alb = df[df['Tuna']=='albacore ']

print(alb.shape)
alb.head()

(43, 2)


,Tuna,Mercury
0,albacore,0.000
1,albacore,0.410
2,albacore,0.820
3,albacore,0.320
4,albacore,0.036


In [70]:
yfin = df[df['Tuna']=='yellowfin']

print(yfin.shape)
yfin.head()

(231, 2)


,Tuna,Mercury
43,yellowfin,0.00
44,yellowfin,0.00
45,yellowfin,0.06
46,yellowfin,0.09
47,yellowfin,0.17


In [84]:
double_tstat(alb['Mercury'], yfin['Mercury'])

var1 and var2 are statistically similar in this regard.
